### Hierin staan alle modellen die gebruikt worden in de lecture notes. Run de imports hieronder om de modellen te gebruiken

In [ ]:
# =================================
# Imports
# =================================
from PyCh import *
from numpy import random
from dataclasses import dataclass
import math

<h1><center> Lecture notes</center></h1>


# A. KIVA model

In [ ]:
# =================================
# Pod definition
# =================================
@dataclass
class Pod:
    id: int = 0

# =================================
# Generator definition
# =================================
@process
def Generator(env, c_out, N):
    for i in range(N):
        x = Pod()
        send = c_out.send(x)
        yield env.execute(send)

# =================================
# Storage definition
# =================================
@process
def Storage(env, c_in, c_out, la):
    while True:
        receive = c_in.receive()
        x = yield env.execute(receive)

        delay = random.exponential(1.0 / la)
        yield env.timeout(delay)

        send = c_out.send(x)
        yield env.execute(send)

# =================================
# Buffer definition
# =================================
@process
def Buffer(env, c_in, c_out):
    xs = [] # list of pods
    while True:
        receiving = c_in.receive()
        sending = c_out.send(xs[0]) if len(xs)>0 else None
        yield env.select(sending, receiving)
        if selected(sending):
            xs = xs[1:]
        if selected(receiving):
            x = receiving.entity
            xs = xs + [x]


# =================================
# Pick definition
# =================================
@process
def Pick(env, c_in, c_out, mu, n):
    for i in range(n):
        receive = c_in.receive()
        x = yield env.execute(receive)

        delay = random.exponential(1.0 / mu)
        yield env.timeout(delay)

        send = c_out.send(x)
        yield env.execute(send)
    print(f"TH = {n / env.now}")

# =================================
# KIVA Model
# =================================
def model():
    N = 1       # Number of robots
    la = 4.0    # average rate / hour for storing and retrieving racks (averages to 15 min per task)
    mu = 20.0   # average rate / hour for picking of racks (averages to 3 min per task)

    env = Environment()
    a = Channel(env)
    b = Channel(env)
    c = Channel(env)
    G = Generator(env, a, N)
    Ss = [Storage(env, a, b, la) for j in range(N)]
    B = Buffer(env, b, c)
    P = Pick(env, c, a, mu, 10000)

    env.run()
    
# =================================
# Main
# =================================
model()


# B. Zero-buffer model

In [ ]:
# =================================
# Job definition
# =================================
@dataclass
class Job:
    id: int = 0

# =================================
# Generator definition
# =================================
@process
def Generator(env, c_out, u):
    i = 0
    while True:
        x = Job(id = i)
        send = c_out.send(x)
        yield env.execute(send)
        
        delay = u()
        yield env.timeout(delay)
        i = i+1

# =================================
# Machine definition
# =================================
@process
def Machine(env, c_in, c_out, u):
    while True:
        receive = c_in.receive()
        x = yield env.execute(receive)

        delay = u()
        yield env.timeout(delay)

        send = c_out.send(x)
        yield env.execute(send)

# =================================
# Exit definition
# =================================
@process
def Exit(env, c_in, n):
    x = Job()
    while x.id < n:
        receive = c_in.receive()
        x = yield env.execute(receive)
    print(f"TH = {x.id / env.now}")

# =================================
# GME Model
# =================================
def model():
    ta = 1.0
    te = 1.0
    n  = 1000

    env = Environment()
    a = Channel(env)
    b = Channel(env)
    G = Generator(env, a, lambda: ta)
    M = Machine(env, a, b, lambda: te)
    E = Exit(env, b, n)

    env.run()

# =================================
# Main
# =================================
model()

# C. Finite Buffer Model

In [ ]:
# =================================
# Job definition
# =================================
@dataclass
class Job:
    id: int = 0

# =================================
# Generator definition
# =================================
@process
def Generator(env, c_out, u):
    i = 0
    while True:
        x = Job(id = i)
        send = c_out.send(x)
        yield env.execute(send)
        
        delay = u()
        yield env.timeout(delay)
        i = i+1

# =================================
# Buffer definition
# =================================
@process
def Buffer(env, c_in, c_out, N):
    xs = []
    while True:
        receiving = c_in.receive() if len(xs)<N else None
        sending = c_out.send(xs[0]) if len(xs)>0 else None
        yield env.select(sending,receiving)
        if selected(sending):
            xs = xs[1:]
        if selected(receiving):
            x = receiving.entity
            xs = xs + [x]       
        
# =================================
# Machine definition
# =================================
@process
def Machine(env, c_in, c_out, u):
    while True:
        receive = c_in.receive()
        x = yield env.execute(receive)

        delay = u()
        yield env.timeout(delay)

        send = c_out.send(x)
        yield env.execute(send)

# =================================
# Exit definition
# =================================
@process
def Exit(env, c_in, n):
    x = Job()
    while x.id < n:
        receive = c_in.receive()
        x = yield env.execute(receive)
    print(f"TH = {x.id / env.now}")

# =================================
# GBME Model
# =================================
def model():
    ta = 1.0
    te = 1.0
    n  = 1000
    N = 10

    env = Environment()
    a = Channel(env)
    b = Channel(env)
    c = Channel(env)
    G = Generator(env, a, lambda: random.exponential(ta))
    B = Buffer(env, a, b,  N)
    M = Machine(env, b, c, lambda: random.exponential(te))
    E = Exit(env, c, n)

    env.run()
    
# =================================
# Main
# =================================
model()

# D. Single Machine Model

In [ ]:
# =================================
# Job definition
# =================================
@dataclass
class Job:
    entrytime: float = 0.0

# =================================
# Generator definition
# =================================
@process
def Generator(env, c_out, u):
    while True:
        x = Job(entrytime = env.now)
        send = c_out.send(x)
        yield env.execute(send)
        
        delay = u()
        yield env.timeout(delay)

# =================================
# Buffer definition
# =================================
@process
def Buffer(env, c_in, c_out):
    xs = []
    while True:
        receiving = c_in.receive()
        sending = c_out.send(xs[0]) if len(xs)>0 else None
        yield env.select(sending,receiving)
        if selected(sending):
            xs = xs[1:]
        if selected(receiving):
            x = receiving.entity
            xs = xs + [x]       
        
# =================================
# Machine definition
# =================================
@process
def Machine(env, c_in, c_out, u):
    while True:
        receive = c_in.receive()
        x = yield env.execute(receive)
        
        send = c_out.send(x)
        yield env.execute(send)

        delay = u()
        yield env.timeout(delay)


# =================================
# Exit definition
# =================================
@process
def Exit(env, c_in, n):
    sumw = 0.0
    for i in range(n):
        receive = c_in.receive()
        x = yield env.execute(receive)
        sumw = sumw + (env.now - x.entrytime)
    print(f"Mean waiting time spent in buffer = {(sumw / n):g}")

# =================================
# GBME Model
# =================================
def model():
    n  = 100000

    env = Environment()
    a = Channel(env)
    b = Channel(env)
    c = Channel(env)
    G = Generator(env, a, lambda: random.uniform(0.0, 2.0))
    B = Buffer(env, a, b)
    M = Machine(env, b, c, lambda: random.uniform(0.0, 1.0))
    E = Exit(env, c, n)

    env.run(until=E)
    
# =================================
# Main
# =================================
model()

# E. Multi machine model

In [ ]:
# =================================
# Job definition
# =================================
@dataclass
class Job:
    entrytime: float = 0.0

# =================================
# Generator definition
# =================================
@process
def Generator(env, c_out, u):
    while True:
        x = Job(entrytime = env.now)
        send = c_out.send(x)
        yield env.execute(send)
        
        delay = u()
        yield env.timeout(delay)

# =================================
# Buffer definition
# =================================
@process
def Buffer(env, c_in, c_out):
    xs = []
    while True:
        receiving = c_in.receive()
        sending = c_out.send(xs[0]) if len(xs)>0 else None
        yield env.select(sending,receiving)
        if selected(sending):
            xs = xs[1:]
        if selected(receiving):
            x = receiving.entity
            xs = xs + [x]       
        
# =================================
# Machine definition
# =================================
@process
def Machine(env, c_in, c_out, u):
    while True:
        receive = c_in.receive()
        x = yield env.execute(receive)
        
        send = c_out.send(x)
        yield env.execute(send)

        delay = u()
        yield env.timeout(delay)


# =================================
# Exit definition
# =================================
@process
def Exit(env, c_in, n):
    sumw = 0.0
    for i in range(n):
        receive = c_in.receive()
        x = yield env.execute(receive)
        sumw = sumw + (env.now - x.entrytime)
    print(f"Mean waiting time spent in buffer = {(sumw / n):g}")

# =================================
# GBME Model
# =================================
def model():
    m = 2
    n = 100000

    env = Environment()
    a = Channel(env)
    b = Channel(env)
    c = Channel(env)
    G = Generator(env, a, lambda: random.exponential(0.111))
    B = Buffer(env, a, b)
    Ms = [ Machine(env, b, c, lambda: random.uniform(0.0, 0.4)) for j in range(m) ]
    E = Exit(env, c, n)

    env.run(until=E)

# =================================
# Main
# =================================
model()

# F. Serial production line

In [ ]:
# =================================
# Job definition
# =================================
@dataclass
class Job:
    entrytime: float = 0.0

# =================================
# Generator definition
# =================================
@process
def Generator(env, c_out, u):
    while True:
        x = Job(entrytime = env.now)
        send = c_out.send(x)
        yield env.execute(send)
        
        delay = u()
        yield env.timeout(delay)

# =================================
# Buffer definition
# =================================
@process
def Buffer(env, c_in, c_out):
    xs = []
    while True:
        receiving = c_in.receive()
        sending = c_out.send(xs[0]) if len(xs)>0 else None
        yield env.select(sending, receiving)
        if selected(sending):
            xs = xs[1:]
        if selected(receiving):
            x = receiving.entity
            xs = xs + [x]       
        
# =================================
# Machine definition
# =================================
@process
def Machine(env, c_in, c_out, u):
    while True:
        receive = c_in.receive()
        x = yield env.execute(c_in.receive())

        send = c_out.send(x)
        yield env.execute(c_out.send(x))

        delay = u()
        yield env.timeout(delay)


# =================================
# Exit definition
# =================================
@process
def Exit(env, c_in, n):
    sumw = 0.0
    for i in range(n):
        receive = c_in.receive()
        x = yield env.execute(receive)
        sumw = sumw + (env.now - x.entrytime)
    return sumw / n

# =================================
# Mline Model
# =================================
def model():
    n = 10000

    env = Environment()
    a = [ Channel(env) for i in range(3)]
    b = [ Channel(env) for i in range(2)]
    G = Generator(env, a[0], lambda: random.exponential(0.5))
    B1 = Buffer(env, a[0], b[0])
    M1 = Machine(env, b[0], a[1], lambda: 0.33)
    B1 = Buffer(env, a[1], b[1])
    M1 = Machine(env, b[1], a[2], lambda: random.uniform(0.0, 0.8))
    E = Exit(env, a[2], n)
    env.run(until=E)
    
    return E.value


# =================================
# Experiment
# =================================
def Experiment():
    m = 10
    phi = 0.0
    sum1 = 0.0
    sum2 = 0.0
    smean = 0.0
    svar = 0.0
    
    for i in range(m):
        phi = model()
        sum1 = sum1 + phi
        sum2 = sum2 + phi*phi
        print(f"Mean flow time in run {i+1:d} is {phi:g}")
        
    smean = sum1 / m
    svar = sum2 / m - smean * smean
    print(f"Mean flow time estimate is {smean:g} +- {1.96 * math.sqrt(svar/m):g}")

# =================================
# Main
# =================================
Experiment()